In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, row_number,when,col,min,max,unix_timestamp,rank, dense_rank,date_format,count,lead,lag,sum,expr,from_utc_timestamp,substring,abs,from_unixtime,to_date,greatest,to_timestamp,lit,concat,split,current_date, concat_ws,regexp_extract,lpad,first, last,date_add,hour,coalesce,floor,window
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("MulRead").getOrCreate()
# spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", sas_token)

In [55]:
folder_path = "/Users/sushantshinde/workspaces/trvi/ALFOMATIC/*.csv"


In [56]:
# Read all CSV files in the folder
windowSpec_csv_filename = Window.partitionBy('FileName').orderBy("FileName")
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(folder_path).withColumn("FileName", input_file_name()).withColumn("join_date", date_format(col("_time"), "yyyy-MM-dd HH:mm:ss"))\
    .withColumn("FileName", input_file_name())\
    .withColumn("file_id", dense_rank().over(Window.orderBy("FileName")))
    # .withColumn("date_only", to_date(col("_start")))\
    # .withColumn("date_id", dense_rank().over(Window.orderBy('date_only')))\
    # .withColumn("FileName", regexp_extract(col("FileName"), r"([^/]+)$", 1)) \
    # .withColumn("file_date", substring(col("FileName"), 1, 8)).withColumn("date_id", concat(lit("ID_"), col("file_date")))\
    # .withColumn("date_id", expr("CAST(SUBSTRING(date_id, 4, LENGTH(date_id)) AS INT)"))
    # .withColumn("_time", from_utc_timestamp(col("_time"), "Asia/Kolkata")).withColumn("_stop", from_utc_timestamp(col("_stop"), "Asia/Kolkata")).withColumn("_start", from_utc_timestamp(col("_start"), "Asia/Kolkata"))\
    # 

    # .withColumn("file_date", substring(col("FileName"), 1, 8))\
#    .withColumn("date_id", concat(lit("ID_"), col("file_date")))\ dinesgji banas 

In [57]:
df2 = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Users/sushantshinde/workspaces/trvi/plc08-11/*.csv").withColumn("join_date", date_format(col("_time"), "yyyy-MM-dd HH:mm:ss"))

In [58]:
df_join = df.alias('df').join(df2.alias('df2'), on="join_date", how="left").select('df.*','df2.Report_Belt1_Speed','df2.Report_Belt2_Speed','df2.Report_Belt2LanceFlow')

In [59]:
# group_df = df_join.groupBy(col('V_G20EMP_N'),col('file_id')).agg(date_format(min(col("_time")).cast("timestamp"),"HH:mm:ss").alias('Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Stop_Time'),max('Report_Belt1_Speed').alias('speed'),max('Report_Belt2_Speed').alias('lower speed'))

In [60]:
# window_spec = Window.partitionBy("V_G20EMP_N", "start_time").orderBy(col('Stop_Time').desc())


# df_result = group_df.alias('df1').join(group_df.alias('df2'), (col('df1.V_G20EMP_N') == col('df2.V_G20EMP_N')) & (col('df1.file_id') == col('df2.file_id') + 1),'left')\
# .select('df1.V_G20EMP_N','df1.file_id','df1.speed','df1.lower speed',when(col("df2.Start_Time").isNotNull(), col("df2.Start_Time")).otherwise(col("df1.Start_Time")).alias('start_time'),'df1.Stop_Time').filter(col('V_G20EMP_N') != 0)\
# .withColumn("row_num", row_number().over(window_spec)).filter(col('row_num') == 1).drop("row_num")


In [61]:
# df_result.orderBy('df1.file_id').show(df_result.count(),truncate=False)

In [62]:
# group_df_end = df_join.groupBy(col('V_G20EMP_N_1'),col('file_id')).agg(date_format(min(col("_time")).cast("timestamp"),"HH:mm:ss").alias('Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Stop_Time')).filter(col('V_G20EMP_N_1') != 0)

In [63]:
# window_spe2 = Window.partitionBy("V_G20EMP_N_1", "start_time").orderBy(col('Stop_Time').desc())

# df_result_end = group_df_end.alias('df1').join(group_df_end.alias('df2'), (col('df1.V_G20EMP_N_1') == col('df2.V_G20EMP_N_1')) & (col('df1.file_id') == col('df2.file_id') + 1),'left')\
# .select('df1.V_G20EMP_N_1','df1.file_id',when(col("df2.Start_Time").isNotNull(), col("df2.Start_Time")).otherwise(col("df1.Start_Time")).alias('start_time'),'df1.Stop_Time')\
# .withColumn("row_num", row_number().over(window_spe2)).filter(col('row_num') == 1).drop("row_num")

In [64]:
# df_result_end.orderBy('file_id').show(df_result_end.count(),truncate=False)

In [65]:
# df_join_cooking_df =  df_result.alias('df1').join(df_result_end.alias('df2'), (col('df1.V_G20EMP_N') == col('df2.V_G20EMP_N_1')) &         
#             (date_format(col("df1.Stop_Time"), "HH:mm") == date_format(col("df2.start_time"), "HH:mm")),'left').select('V_G20EMP_N','df1.start_time','df2.Stop_Time','speed','df1.lower speed','df1.file_id')

In [66]:
# df_final_test = df_join_cooking_df.withColumn("upper_belft_stop", date_format(to_timestamp(col("Stop_Time"), "HH:mm:ss") + expr("INTERVAL 30 MINUTES"), "HH:mm:ss")
# ).withColumn("lower_belft_stop", date_format(to_timestamp(col("upper_belft_stop"), "HH:mm:ss") + expr("INTERVAL 60 MINUTES"), "HH:mm:ss")
# )

In [67]:
# when(col("df2.Cooking_Stop_Time").isNotNull(), col("df2.Cooking_Stop_Time")).otherwise(col("df1.Cooking_Stop_Time"))

In [68]:
# group_df.orderBy('file_id').show(group_df.count(),truncate=False)

In [69]:
# df2 = spark.read.format("csv") \
#     .option("header", "true") \
#     # .option("inferSchema", "true") 
#     .load("/Users/sushantshinde/workspaces/trvi/plc08-11/*.csv").withColumn("join_date", date_format(col("_time"), "yyyy-MM-dd HH:mm:ss"))

In [70]:
# df_join = df.alias('df').join(df2.alias('df2'), on="join_date", how="left").select('df.*','df2.Report_Belt1_Speed','df2.Report_Belt2_Speed','df2.Report_Belt2LanceFlow')

In [71]:
# df_join.show(truncate=False)

In [72]:
# blob_path = '/Users/sushantshinde/workspaces/trvi/output'
# df_final_test.repartition(1).orderBy('file_id') \
#     .write \
#     .mode("overwrite") \
#     .option("header", True) \
#     .csv(blob_path)

In [73]:
# blob_path = '/Users/sushantshinde/workspaces/trvi/output'
# df.repartition(1).orderBy('_time') \
#     .write \
#     .mode("overwrite") \
#     .option("header", True) \
#     .csv(blob_path)

In [74]:
# upper_df = df_join.select('_start','_stop','_time','_measurement','V_F03LENG','AI_D50B66','V_G20EMP_N','V_G20EMP_N_1','V_G23SPEED_N','V_G23SPEED_N_1','Report_Belt1_Speed')\
# .withColumn('volumn',col('V_F03LENG') * col('AI_D50B66') * 2)

In [75]:
# lower_df= df_join.select('_start','_stop','_time','_measurement','V_F07LENG','V_G01HGHT1','V_G02SALT1','V_H80SETP','V_G20EMP_N','V_G20EMP_N_1','V_G23SPEED_N','V_G23SPEED_N_1','Report_Belt2_Speed')\
# .withColumn('volumn',col('V_F07LENG') * col('V_G01HGHT1') * 2)

In [76]:
# blob_path = '/Users/sushantshinde/workspaces/trvi/output'
# upper_df.repartition(1).orderBy('_time') \
#     .write \
#     .mode("overwrite") \
#     .option("header", True) \
#     .csv(blob_path)

In [77]:
# import shutil
# import glob

# output_folder = "/Users/sushantshinde/workspaces/trvi/output"
# csv_files = glob.glob(f"{output_folder}/part-*.csv")

# if csv_files:
#     shutil.move(csv_files[0], "/Users/sushantshinde/workspaces/trvi/output_csv/upper_belt-data.csv")

In [ ]:
df_ids = df_join.withColumn('VatNo1',when((col('V_G20EMP_N') == 1) | (col('V_G20EMP_N_1') == 1), col('_time')))\
                .withColumn('VatNo2',when((col('V_G20EMP_N') == 2) | (col('V_G20EMP_N_1') == 2), col('_time')))\
                .withColumn('VatNo3',when((col('V_G20EMP_N') == 3) | (col('V_G20EMP_N_1') == 3), col('_time')))\
                .withColumn('VatNo4',when((col('V_G20EMP_N') == 4) | (col('V_G20EMP_N_1') == 4), col('_time')))\
                .withColumn('VatNo5',when((col('V_G20EMP_N') == 5) | (col('V_G20EMP_N_1') == 5), col('_time')))\
                .withColumn('VatNo6',when((col('V_G20EMP_N') == 6) | (col('V_G20EMP_N_1') == 6), col('_time')))\
                .withColumn('VatNo7',when((col('V_G20EMP_N') == 7) | (col('V_G20EMP_N_1') == 7), col('_time')))

In [79]:
# # Step 1: Assign row numbers and batch IDs
# window_spec = Window.orderBy("_time")
# df2 = df2.withColumn("row_num", row_number().over(window_spec))
# df2 = df2.withColumn("batch_id", floor((col("row_num") - 1) / 30))  # Every 30 records = 1 batch

# # Step 2: Get the batch start time (Earliest _time in each batch)
# batch_window = Window.partitionBy("batch_id").orderBy("_time").rowsBetween(Window.unboundedPreceding, Window.currentRow)
# df2 = df2.withColumn("batch_start_time", min("_time").over(batch_window))

# # Step 3: Extract HHmmss time window
# df2 = df2.withColumn("time_window", date_format(col("batch_start_time"), "HHmmss"))

# # Step 4: Generate upper belt ID
# vat_columns = [f"VatNo{i}" for i in range(1, 8)]
# df2 = df2.withColumn("id", 
#                    concat_ws("", 
#                              *[when(col(vat).isNotNull(), lit(f"vat{i}_"))
#                              .otherwise(lit("")) for i, vat in enumerate(vat_columns, 1)],
#                              col("time_window")
#                    ))
# df2 = df2.withColumn("upper_belt_id", concat_ws("_", col("id"), col("batch_id")))

# # Step 5: Calculate the lower belt start time (exactly 30 minutes after upper belt start)
# df2 = df2.withColumn("lower_belt_start_time", col("batch_start_time") + expr("INTERVAL 30 MINUTES"))

# # Step 6: Find the row where _time matches the calculated lower belt start time
# df_lower = df2.select("_time", "upper_belt_id").withColumnRenamed("_time", "lower_belt_start_time")

# df_lower = df_lower.select("lower_belt_start_time", col("upper_belt_id").alias("temp_lower_belt_id"))

# df2 = df2.join(df_lower, on="lower_belt_start_time", how="left")

# df2 = df2.withColumn("lower_belt_id", when(col("temp_lower_belt_id").isNotNull(), col("temp_lower_belt_id")).otherwise(None)) \
#          .drop("temp_lower_belt_id")
# # Show the result
# df2.select("_time", "upper_belt_id", "lower_belt_start_time", "lower_belt_id").show(50, False)

In [80]:
window_spec = Window.orderBy("_time")
df_ids = df_ids.withColumn("row_num", row_number().over(window_spec))


In [81]:
df_ids = df_ids.withColumn("batch_id", floor((col("row_num") - 1) / 31))
df_ids = df_ids.withColumn("time_window", date_format(col("_time"), "HHmmss"))

vat_columns = [f"VatNo{i}" for i in range(1, 8)]  # vat_1_date to vat_7_date
df2 = df_ids.withColumn("id", 
                   concat_ws("", 
                             *[when(col(vat).isNotNull(), lit(f"vat{i}_"))
                             .otherwise(lit("")) for i, vat in enumerate(vat_columns, 1)],
                             col("time_window")
                   ))

In [82]:
# df_ids.select('VatNo1','VatNo2','VatNo3','VatNo4','VatNo5','VatNo6','VatNo7','_time','time_window','batch_start_time','batch_time_str','id').show(50,truncate=False)

In [83]:
# Remove empty strings from the ID
df2 = df2.withColumn("id", 
                   concat_ws("", 
                             *[col("id").substr(i, 4) for i in range(1, 29, 4)]
                   ))

df2 = df2.withColumn("upper_belt_id", concat_ws("_", col("id"), col("batch_id")))


In [84]:
# Calculate lower belt start time (30 minutes after upper belt start time)
df2 = df2.withColumn("lower_belt_time", expr("cast(_time as timestamp) + INTERVAL 30 MINUTES"))

# Generate lower belt ID
df2 = df2.withColumn("lower_belt_time_window", date_format(col("lower_belt_time"), "HHmmss"))
window_spec = Window.orderBy("_time")

df2 = df2.withColumn("lower_belt_id", 
                   concat_ws("", 
                             *[when(col(vat).isNotNull(), lit(f"vat{i}_"))
                             .otherwise(lit("")) for i, vat in enumerate(vat_columns, 1)],
                             col("lower_belt_time_window")
                   ))# df2.select("_time", "upper_belt_id", "lower_belt_time", "lower_belt_id").show(truncate=False)

df2 = df2.withColumn("lower_belt_id", 
                   concat_ws("", 
                             *[col("lower_belt_id").substr(i, 4) for i in range(1, 29, 4)]
                   ))

df2 = df2.withColumn("lower_belt_id", concat_ws("_", col("lower_belt_id"), col("batch_id")))




df2 = df2.withColumn("lower_belt_id", col("lower_belt_id"))


# Show the result
# df2.select("_time", "upper_belt_id", "lower_belt_time", "lower_belt_id").show(100, truncate=False)


In [85]:
df2 = df2.withColumn("vat_1", split(col("upper_belt_id"), "_").getItem(0))

df2 = df2.withColumn("lower_belt_window", window(col("lower_belt_time"), "15 seconds"))
df2 = df2.withColumn("vat_1", split(col("upper_belt_id"), "_").getItem(0))
# Step 3: Generate the `HHmmss` part from `lower_belt_time`
# df2 = df2.withColumn("time_part", date_format(col("lower_belt_time"), "HHmmss"))

# Step 4: Extract `batch_id` from `upper_belt_id`
df2 = df2.withColumn("batch_id", split(col("upper_belt_id"), "_").getItem(2))



# Step 5: Construct `lower_belt_id` by combining `vat_1`, `time_part`, and `batch_id`
df2 = df2.withColumn("lower_belt_id", 
                   concat_ws("_", 
                             col("vat_1"),  # Extract vat_1 from upper_belt_id
                             date_format(col("lower_belt_window.start"), "HHmmss"),  # Window start time
                             col("batch_id")  # Include batch_id
                   ))

df2 = df2.drop("vat_1", "lower_belt_window")


In [ ]:
df2 = df2.withColumn("lower_belt_time_exit_time", expr("cast(lower_belt_time as timestamp) + INTERVAL 60 MINUTES"))


In [93]:
window_spec = Window.orderBy("_time")
df2 = df2.withColumn("row_num", row_number().over(window_spec))


In [94]:
df2 = df2.withColumn("block_1", 
                   when(col("row_num") % 2 == 1, col("lower_belt_id"))  # Odd rows go to block_1
                   .otherwise(None))

df2 = df2.withColumn("block_2", 
                   when(col("row_num") % 2 == 0, col("lower_belt_id"))  # Even rows go to block_2
                   .otherwise(None))
df2 = df2.drop("row_num")


In [87]:
# df2 = df2.withColumn("lower_belt_time", col("_time") + expr("INTERVAL 30 MINUTES"))

# # Generate lower belt ID
# df2 = df2.withColumn("lower_belt_time_window", date_format(col("lower_belt_time"), "HHmmss"))
# df2 = df2.withColumn("lower_belt_id", 
#                    concat_ws("", 
#                              *[when(col(vat).isNotNull(), lit(f"vat{i}_"))
#                              .otherwise(lit("")) for i, vat in enumerate(vat_columns, 1)],
#                              col("lower_belt_time_window")
#                    ))

# # Remove empty strings from the lower belt ID
# df2 = df2.withColumn("lower_belt_id", 
#                    concat_ws("", 
#                              *[col("lower_belt_id").substr(i, 4) for i in range(1, 29, 4)]
#                    ))

# # Add lower belt ID
# df2 = df2.withColumn("lower_belt_id", concat_ws("_", col("lower_belt_id"), col("batch_id")))

# # Assign lower belt ID to the corresponding row
# df2 = df2.withColumn("lower_belt_id", 
#                    when(col("_time") == col("lower_belt_time"), col("lower_belt_id"))
#                    .otherwise(None))

# # Show the result
# df2.select("_time", "upper_belt_id", "lower_belt_time", "lower_belt_id").show(truncate=False)

In [88]:
# df2.select('VatNo1','VatNo2','VatNo3','VatNo4','VatNo5','VatNo6','VatNo7','_time','time_window','batch_start_timAe','batch_time_str','id','upper_belt_id').show(50,truncate=False)

In [95]:
blob_path = '/Users/sushantshinde/workspaces/trvi/output'
df2.repartition(1).orderBy('_time') \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(blob_path)

25/03/07 22:45:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/07 22:45:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/07 22:45:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/07 22:45:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/07 22:45:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/07 22:45:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/07 2

25/03/07 23:56:11 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1082183 ms exceeds timeout 120000 ms
25/03/07 23:56:11 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/07 23:56:14 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$